In [9]:
!rm -r /content/train
!rm -r /content/test

rm: cannot remove '/content/train': No such file or directory
rm: cannot remove '/content/test': No such file or directory


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!wget --no-check-certificate \
        https://www.dropbox.com/s/8u2k79tuqmwrwi8/train.zip \
       -O /tmp/train.zip

--2021-02-14 13:04:45--  https://www.dropbox.com/s/8u2k79tuqmwrwi8/train.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/8u2k79tuqmwrwi8/train.zip [following]
--2021-02-14 13:04:45--  https://www.dropbox.com/s/raw/8u2k79tuqmwrwi8/train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc533ceeae4131262cb5c7c764c3.dl.dropboxusercontent.com/cd/0/inline/BI6UPqLE8uGFBXhMarRkJ30q8JWz3YEmRNRgGnUzPUovF_sC6LXoN76ffUADpIY9Pm25oVcWZ2XL7zdjBF4MTeNmQ6xkebRLnLBW_VOislTKtW3XgQyPYlPGlRAp9Sj1ssc/file# [following]
--2021-02-14 13:04:46--  https://uc533ceeae4131262cb5c7c764c3.dl.dropboxusercontent.com/cd/0/inline/BI6UPqLE8uGFBXhMarRkJ30q8JWz3YEmRNRgGnUzPUovF_sC6LXoN76ffUADpIY9Pm25oVcWZ2XL7zdjBF4MTeNmQ6xkebRLnLBW_VOislTKtW3XgQyPYlPG

In [12]:
!wget --no-check-certificate \
        https://www.dropbox.com/s/pnipjr7brjz1pm5/test.zip \
       -O /tmp/test.zip

--2021-02-14 13:04:50--  https://www.dropbox.com/s/pnipjr7brjz1pm5/test.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pnipjr7brjz1pm5/test.zip [following]
--2021-02-14 13:04:50--  https://www.dropbox.com/s/raw/pnipjr7brjz1pm5/test.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf00a867f6186b2df263acb2eec.dl.dropboxusercontent.com/cd/0/inline/BI5Abck7PnRR0QrBd1iohXlfxCF4rrUWzdqVtY2bUvRJxAPizL_rGqMK498zeRlpM7rqCog8otP7oyPLnOFwe2q5oSJqvtVBz26yoa7uyb_BoGHDUWIFb2KYG6GwRUdGqgk/file# [following]
--2021-02-14 13:04:50--  https://ucf00a867f6186b2df263acb2eec.dl.dropboxusercontent.com/cd/0/inline/BI5Abck7PnRR0QrBd1iohXlfxCF4rrUWzdqVtY2bUvRJxAPizL_rGqMK498zeRlpM7rqCog8otP7oyPLnOFwe2q5oSJqvtVBz26yoa7uyb_BoGHDUWIFb2KYG6G

In [13]:
import zipfile

with zipfile.ZipFile("//tmp/train.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")
zip_ref.close()

with zipfile.ZipFile("//tmp/test.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")
zip_ref.close()

In [14]:
TRAIN_DIR = '/content/train'
TEST_DIR = '/content/test'

In [15]:
BATCH_SIZE = 32

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=.2)

train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='training'
)

val_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True, subset='validation'
)

Found 13210 images belonging to 32 classes.
Found 3286 images belonging to 32 classes.


In [18]:
test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_directory(
    directory=TEST_DIR, class_mode='categorical', target_size=(224, 224),
    batch_size=BATCH_SIZE, shuffle=True
)

Found 4142 images belonging to 32 classes.


In [19]:
MAP_CHARACTERS = {v:k for k, v in train_generator.class_indices.items()}

In [20]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [21]:
tf.keras.backend.clear_session()

In [22]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(.1),
    tf.keras.layers.Dense(len(MAP_CHARACTERS), activation='softmax')
])

In [23]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
train_generator.samples

13210

In [25]:
model.fit(train_generator,
          validation_data=val_generator,
          steps_per_epoch=train_generator.samples / BATCH_SIZE,
          epochs=10)

Epoch 1/10
412/412 [==============================] - 198s 464ms/step - loss: 3.5019 - accuracy: 0.2053 - val_loss: 2.0853 - val_accuracy: 0.4410
Epoch 2/10
412/412 [==============================] - 190s 461ms/step - loss: 1.8526 - accuracy: 0.4795 - val_loss: 1.8394 - val_accuracy: 0.5100
Epoch 3/10
412/412 [==============================] - 191s 463ms/step - loss: 1.5236 - accuracy: 0.5659 - val_loss: 1.6869 - val_accuracy: 0.5338
Epoch 4/10
412/412 [==============================] - 191s 463ms/step - loss: 1.2996 - accuracy: 0.6254 - val_loss: 1.6087 - val_accuracy: 0.5603
Epoch 5/10
412/412 [==============================] - 190s 460ms/step - loss: 1.1294 - accuracy: 0.6739 - val_loss: 1.6981 - val_accuracy: 0.5764
Epoch 6/10
412/412 [==============================] - 189s 458ms/step - loss: 0.9836 - accuracy: 0.7196 - val_loss: 1.6434 - val_accuracy: 0.5928
Epoch 7/10
412/412 [==============================] - 189s 458ms/step - loss: 0.8888 - accuracy: 0.7421 - val_loss: 1.6081 -

In [26]:
model.evaluate(test_generator)

130/130 [==============================] - 13s 98ms/step - loss: 1.1085 - accuracy: 0.7202


[1.1084773540496826, 0.7201834917068481]

In [27]:
import os
save_path = os.path.join("//content/drive/MyDrive/simpsonsnet/1/")
tf.saved_model.save(model, save_path)

INFO:tensorflow:Assets written to: //content/drive/MyDrive/simpsonsnet/1/assets


In [28]:
save_path = os.path.join("//content/drive/MyDrive/simpsonsnet.h5")
model.save(save_path)